In [36]:
# load necessary python libraries

import pandas as pd

In [37]:
# read the cvs file into Pandas for the Kepler Object data

x_data = pd.read_csv('x_parms.csv')
x_data.head()


,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [38]:
# read y values into Pandas for Kepler classification data

y_data = pd.read_csv('y_value.csv')
y_data.head()

,koi_disposition
0,CONFIRMED
1,CONFIRMED
2,FALSE POSITIVE
3,FALSE POSITIVE
4,CONFIRMED


In [39]:
# Instantiate Decision Tree model
from sklearn import tree


In [40]:
# define training and test data for the Decision Tree model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, random_state=5)

In [41]:
# Create and score Decision Tree model
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8417200365965233

## DCF Classification Score: 0.841

In [46]:
# evaluate feature importance from X parameters
features_df = pd.DataFrame(zip(x_data.columns, clf.feature_importances_))
# sort by feature importance 
features_df = features_df.sort_values(by=[1], ascending=False)


In [43]:
# drop all variables less than 0.02 feature importance
x_filtered_df = features_df[features_df[1]>=0.02]
x_filtered_df

,0,1
0,koi_fpflag_nt,0.193889
2,koi_fpflag_co,0.179764
1,koi_fpflag_ss,0.170684
26,koi_model_snr,0.140486
3,koi_fpflag_ec,0.034405
10,koi_impact,0.025782
39,koi_kepmag,0.023796


In [44]:
# create new dataframe from reduced x_parameters decision tree output
x_filtered_df = x_data[['koi_fpflag_nt', 'koi_fpflag_co', 'koi_fpflag_ss','koi_model_snr', 'koi_fpflag_ec', 'koi_impact']]
x_filtered_df.head()

,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ss,koi_model_snr,koi_fpflag_ec,koi_impact
0,0,0,0,35.8,0,0.146
1,0,0,0,25.8,0,0.586
2,0,0,1,76.3,0,0.969
3,0,0,1,505.6,0,1.276
4,0,0,0,40.9,0,0.701


In [45]:
# write out new x parameters data file
x_filtered_df.to_csv("x_dcf_filt.csv", index=False, header=True)